## 필요한 모듈 불러오기

In [3]:
import numpy as np
import pandas as pd
import json
from gensim.models import Word2Vec
from scipy.sparse import csr_matrix
from scipy import sparse
from implicit.evaluation import  *
from implicit.als import AlternatingLeastSquares
import os; os.environ['KMP_DUPLICATE_LIB_OK']='True'
from collections import Counter
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.rcParams["figure.figsize"] = (20, 10)
plt.style.use('ggplot')

## 데이터 업로드
1. 곡 메타데이터
2. Trainset
3. Testset

In [4]:
# 1. 메타데이터
song_meta = pd.read_json('song_meta.json', typ = 'frame')
# song_meta.head()

In [5]:
# 2. trainset
playlist_df = pd.read_json('train.json', typ = 'frame')
# playlist_df.head()

In [6]:
# 3. testset
test_df = pd.read_csv('testset.csv', index_col=0)
# test_df.head()

## 미리 저장해둔 두개의 매트릭스 사용하기 
- [Matrix-1] : 태그 유사도 행렬(태그를 벡터화한 모델)
- [Matrix-2] : 플레이리스트(유저) - 곡(아이템) 에 대한 선호도 R 행렬

In [7]:
# Matrix-1 : 저장한 모델 불러오기
model = Word2Vec.load('tag2vec.model')

# Matrix-2 : R 행렬의 데이터 불러오기
R = sparse.load_npz("R_data.npz")

# initialize a model(모델 정의)
model_ALS = AlternatingLeastSquares(factors=128, regularization=0.08, use_gpu=False)

# train the model on a sparse matrix of item/user/confidence weights
model_ALS.fit(R)

# 유저-잠재요인 / 잠재요인-아이템 행렬에 대한 수치
user_factors = model_ALS.user_factors
item_factors = model_ALS.item_factors.T
print(user_factors.shape); print(item_factors.shape)


(707989, 128)
(128, 153429)


## 추천 알고리즘 메소드

In [8]:
# 듣고 있는 곡의 태그와 유사한 태그 찾기(word='___')
def tags_for_rec(t_model, tag_name, top_n):
    
    pred_tags = t_model.wv.similar_by_word(word=tag_name, topn=top_n)
    tag_list = [tag for tag, _ in pred_tags]
    return tag_list

In [9]:
# 태그에서 플레이 리스트를 추천하는 메소드 만들기
def plst_id_for_rec(data, tags_list):
    plst_id_for_rec = []
    
    for index, value in enumerate(data[['id','tags']].values):
        # print(value[0])    
        for tag in tags_list:
            # print(tag_list)
            if tags_list[1] in value[1]: 
                plst_id_for_rec.append(value[0])
    # 후보 태그에 대한 추천될 플레이리스트 id 후보군이 형성된다.
    return plst_id_for_rec # 중복 언급 허용

In [10]:
# 플레이리스트 후보군에서 추천하는 노래 ID 후보군을 추출하는 메소드
def songs_for_rec(model, playlist_id, r, n):
    # plst_id_for_rec = list(set(plst_id_for_rec))
    songs_for_rec = []
    
    for idx in range(len(playlist_id)):
        # recommend items for a user
        recommendations = model.recommend(
                            userid = playlist_id[idx],
                            user_items = r,
                            N = n
                            )
        
        for song_id, rate in recommendations:
            # print(song_id)
            songs_for_rec.append(song_id)
            
    return songs_for_rec

In [11]:
# 만들어진 음악 후보군에서 최종으로 추천할 노래 생성(행렬1, 2 모두 적용)
def music_recommend(data, model, item_id, song_for_rec):
    
    answer = []
    recomm = model.similar_items(itemid=item_id)
    count_songs = Counter(song_for_rec)
    # 후보군에서 반복언급된 TOP-3 노래 추출
    max_values = sorted(count_songs.values(), reverse=True)[1:4]
    
    
    ## 우선순위 1위 : 선호관계와 태그유사도에서 중복언급되는 곡id 확인
    
    # 1. 곡을 선호하는 유저 중 비슷한 유형의 TOP-3 유저id 추출
    recommended_id = []
    for idx,_ in enumerate(recomm):
        if idx <= 2:
            recommended_id.append(recomm[idx][0])
        else:
            break
    
    # 2. 추출된 유저id의 플레이리스트 곡id 추출
    songs_id = []
    for pl_id in recommended_id:
        tmp = data[data['id']==pl_id]['songs'].tolist()
        songs_id += tmp[0]
    
    # 3. 이전에 태그유사도에서 추출한 song_for_rec과 교집합 추출
    songs_id = set(songs_id)
    intersect = list(songs_id.intersection(set(song_for_rec)))
    answer += intersect
    
    
    ## 우선순위 2위 : 디폴트로 6곡 추가
    
    # 1. 선호관계 유사도에서 추천곡 3개 디폴트
    songs_count = Counter(songs_id)
    tmpt = [song_id for song_id, _ in songs_count.most_common(3)]
    answer += tmpt
    
    # 2. 태그관계 유사도에서 추천곡 3개 디폴트
    for key in list(count_songs.keys()):       
        if count_songs[key] == max_values :
            answer.append(key)
    # answer.sort()
        
    return answer

In [12]:
# 태그,곡 id만 입력하면 위의 메소드들이 다 실행되는 프로세스
def recommendation_process(tag_model, r_model, train, tag, song_id):
    
    global tags_for_rec, plst_id_for_rec, songs_for_rec, music_recommend
    
    # 1. 태그 유사도 찾기
    tags_for_rec_result = tags_for_rec(t_model=tag_model, tag_name=tag, top_n=3)

    # 2. 태그를 가진 플레이리스트 찾기
    plst_id_for_rec_result = plst_id_for_rec(data=train, tags_list=tags_for_rec_result)

    # 3. 플레이리스트 중 가장 적합한 모델 찾기
    songs_for_rec_result = songs_for_rec(model=r_model,
                                         playlist_id = plst_id_for_rec_result,
                                         r = R,
                                         n = 1)

    # 4. 생성된 후보군으로 최종 모델 찾기
    music_recommend_result = music_recommend(data=train, model=r_model, item_id=song_id, song_for_rec=songs_for_rec_result)
    
    return music_recommend_result